## feature_v4  
feature_v1から移植。  
TXT35成形板。細かい見落としを修正、新しいtraindataを作成。
TXT35について：  
入線順位、確定順位両方あり、レースIDを新に。前走上がり3F順を追加

In [1]:
import numpy as np
import pandas as pd
import math
import pickle
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.simplefilter('ignore')

In [2]:
pickle_dir = "F:/TARGET frontier/TXT35/pickle_data/"
le_save = True
model_save = True

In [3]:
data_dir = "F:/TARGET frontier/TXT35/"
sample_data = pd.DataFrame([])
for i in range(2013,2018+1):
    data_name = "train"+str(i)
    res = pd.read_csv(data_dir+data_name+".csv",encoding="shift-jis")#,na_values=['--']
    sample_data = pd.concat([sample_data,res])
    
#リークしないように慎重に！
data_name = "predict2019"
res = pd.read_csv(data_dir+data_name+".csv",encoding="shift-jis")#,na_values=['--']
predict_len = len(res)
sample_data = pd.concat([sample_data,res])

tail = "fe3"

In [4]:
pd.set_option('display.max_columns', 100)
sample_data.head(2)

,レースID(新),場所,芝・ダ,距離,コース区分,クラス名,馬場状態,出走頭数,騎手,枠番,性別,年齢,キャリア,斤量,馬体重,馬体重増減,間隔,休み明け〜戦目,種牡馬,父タイプ名,多頭出し,所属,調教師,入線順位,確定着順,着差,単勝オッズ,複勝オッズ下限,複勝オッズ上限,複勝配当,前走レースID(新),前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走出走頭数,替,前騎手,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走上り3F順,前走RPCI,前PCI,前好走,前走入線順位,前走確定着順,前走着差タイム,前走PCI3,前走単勝オッズ
0,201312230605091207,中山,芝,1200,A,1000万,良,16,Ｍ．デム,4,牡,3,8,56,494,4.0,3.0,5.0,グラスワンダー,ロイヤルチャージャー系,NaN,(美),鹿戸雄一,1,1,-0.1,1.7,1.1,1.1,110.0,201311300605011206,中山,芝,1200.0,1000万,良,16.0,*,三浦皇成,3.0,先行,56,34.8,34.5,5.0,48.9,50.9,*,2.0,2.0,0.1,51.47,1.9
1,201312230605091203,中山,芝,1200,A,1000万,良,16,北村宏司,2,牡,3,12,56,466,0.0,3.0,4.0,フジキセキ,ロイヤルチャージャー系,*,(栗),清水久詞,2,2,0.1,39.1,4.9,10.8,570.0,201311300605011215,中山,芝,1200.0,1000万,良,16.0,NaN,北村宏司,8.0,後方,56,35.6,34.2,2.0,48.9,54.1,*,7.0,7.0,0.6,51.47,41.3


In [5]:
drop_cols = ["コース区分","多頭出し"]
train_data = sample_data.drop(drop_cols,axis=1)
train_data.shape

(314342, 51)

In [6]:
class Feature_engineering():
    def __init__(self,df):
        self.df = df
        self.l = len(df)
        
    def labelencoding(self,col):
        le = LabelEncoder()
        le.fit(self.df[col])
        self.df[col] = le.transform(self.df[col])
        if le_save:
            with open(pickle_dir+col+tail+'.pickle', mode='wb') as fp:
                pickle.dump(le, fp)
    
    def more_than_double_le(self,lst):
        a,b = lst[0],lst[1]
        le = LabelEncoder()
        le.fit(pd.concat([self.df[a],self.df[b]]))
        self.df[a] = le.transform(self.df[a])
        self.df[b] = le.transform(self.df[b])
        if le_save:
            with open(pickle_dir+a+b+tail+'.pickle', mode='wb') as fp:
                pickle.dump(le, fp)
        
    def handle_nan(self,col):
        self.df.fillna({col:"--"},inplace=True)
        
    def fill_nan(self,col):
        self.df.fillna({col:"0"},inplace=True)
    
    def handle_mark(self,col):
        self.df[col] = self.df[col].str.replace("△","")
        self.df[col] = self.df[col].str.replace("▲","")
        self.df[col] = self.df[col].str.replace("☆","")
        #["△","▲","☆"]
    
    def handle_class(self,col):
        dic_class = {"500万":"1勝","1000万":"2勝","1600万":"3勝","OP(L)":"ｵｰﾌﾟﾝ","重賞":"Ｇ３","(不明)":"--"}
        self.df[col].replace(dic_class,inplace=True)
        
    def handle_specific_nan(self,col):
        dic_specific = {"----":np.nan}
        self.df[col].replace(dic_specific,inplace=True)
    
    def drop_row(self,col):
        self.df.dropna(inplace=True,subset = [col])
        self.df.reset_index(drop=True,inplace=True)
    
    def handle_abs(self,col):
        self.df[col] = self.df[col].astype("float64")
        self.df[col] = self.df[col].where(self.df[col]>0 , 0) #0より小さいなら0を入れてやる
    
    def set_validation(self,y_col,ozz_col):
        self.df[ozz_col] = self.df[ozz_col].astype("float64")
        self.l = len(self.df)
        res = np.array([0.0]*self.l)
        
        for i in range(self.l):
            res[i] = (1/math.exp(3*self.df[y_col][i]))*(1/math.exp(1/self.df[ozz_col][i]*5)) #2/math.exp(1/i*3) min(50,self.df[ozz_col][i]) #1/math.exp(3*self.df[y_col][i])
        self.df[y_col] = np.round(res,5)
    
    #わざと着差をいじる
    def essence(self,y_col,ozz_col):
        res = []
        for i,j in zip(self.df[y_col],self.df[ozz_col]):
            if i == 0 and 9 <= j <= 13:
                res.append(i-0.3)
            else:
                res.append(i)
        self.df[y_col] = pd.DataFrame(res)
    
    def ranking_and_number(self,ID_col):
        ranking = []
        number = []
        num = 0
        rank = 1
        res = ""
        for i,j in enumerate(self.df[ID_col]):
            j = str(j)
            if i == 0:
                ranking.append(rank)
                rank += 1
                res = j[:-2]
                num += 1
                continue
            if res != j[:-2]:
                res = j[:-2]
                number += [num]*num
                rank = 1
                ranking.append(rank)
                rank += 1
                num = 1
            else:
                num += 1
                ranking.append(rank)
                rank += 1
        number += [num]*num
        print(len(number))
        print(len(ranking))
        self.df = pd.concat([self.df,pd.Series(ranking,name="順位"),pd.Series(number,name="頭数")],axis=1)
        
    def basyo_kyori_sibadart(self,basyo,kyori,sibadart):
        self.l = len(self.df)
        res = []
        for i in range(self.l):
            b = self.df[basyo][i]
            k = self.df[kyori][i]
            s = self.df[sibadart][i]
            if any(j == np.nan for j in [b,k,s]):
                res.append(np.nan)
            else:
                res.append(str(b)+str(k)+str(s))
        le = LabelEncoder()
        le.fit(res)
        res = le.transform(res)
        name = basyo+"距離芝ダ"
        res = pd.Series(res,name=name)
        self.df =  pd.concat([self.df,res],axis=1)
        if le_save:
            with open(pickle_dir+name+tail+'.pickle', mode='wb') as fp:
                pickle.dump(le, fp)
        
    def categorical(self,nyusen,ozz): #単勝オッズ15倍以上で3着以内なら1,そうでないなら0
        self.l = len(self.df)
        res = []
        for i in range(self.l):
            if self.df[ozz][i] > 1.8 and self.df[nyusen][i] <= 3:
                res.append(1)
                
            else:
                res.append(0)
        res = pd.Series(res,name='ycategory')
        self.df =  pd.concat([self.df,res],axis=1)
                
        


In [7]:
fe = Feature_engineering(train_data)

In [8]:
nan_cols = ["父タイプ名","前走場所","前芝・ダ","前クラス名","前走馬場状態","前走脚質","前好走","前騎手","替"]
for i in nan_cols:
    print("processing:",i)
    fe.handle_nan(i) #'--'埋めする

processing: 父タイプ名
processing: 前走場所
processing: 前芝・ダ
processing: 前クラス名
processing: 前走馬場状態
processing: 前走脚質
processing: 前好走
processing: 前騎手
processing: 替


In [9]:
fill_nan_with_0 = ["複勝配当","休み明け〜戦目"]
for i in fill_nan_with_0:
    print("processing:",i)
    fe.fill_nan(i)

processing: 複勝配当
processing: 休み明け〜戦目


In [10]:
mark_cols = ["斤量","前走斤量"]
for i in mark_cols:
    print("processing:",i)
    fe.handle_mark(i)

processing: 斤量
processing: 前走斤量


In [11]:
handle_cls = ["クラス名","前クラス名"]
for i in handle_cls:
    print("processing:",i)
    fe.handle_class(i)

processing: クラス名
processing: 前クラス名


In [12]:
double_le = [["場所","前走場所"],["芝・ダ","前芝・ダ"],["馬場状態","前走馬場状態"],["クラス名","前クラス名"],["騎手","前騎手"]]
for i in double_le:
    print("processing:",i)
    fe.more_than_double_le(i)

processing: ['場所', '前走場所']
processing: ['芝・ダ', '前芝・ダ']
processing: ['馬場状態', '前走馬場状態']
processing: ['クラス名', '前クラス名']
processing: ['騎手', '前騎手']


In [13]:
cat_cols = ["性別","種牡馬","父タイプ名","前走脚質","前好走","所属","調教師","替"]
for i in cat_cols:
    print("processing:",i)
    fe.labelencoding(i)

processing: 性別
processing: 種牡馬
processing: 父タイプ名
processing: 前走脚質
processing: 前好走
processing: 所属
processing: 調教師
processing: 替


In [14]:
specific_nan = ["前走着差タイム","着差"]
for i in specific_nan:
    print("processing",i)
    fe.handle_specific_nan(i)

processing 前走着差タイム
processing 着差


In [15]:
drop_na = ["単勝オッズ"]
for i in drop_na:
    print("processing",i)
    fe.drop_row(i)

processing 単勝オッズ


In [16]:
handle_abs = ["前走着差タイム","着差"]
for i in handle_abs:
    print("processing:",i)
    fe.handle_abs(i)

processing: 前走着差タイム
processing: 着差


In [17]:
"""essence = [["着差","単勝オッズ"]]
for i,j in essence:
    print("processing:",i,j)
    fe.essence(i,j)"""

'essence = [["着差","単勝オッズ"]]\nfor i,j in essence:\n    print("processing:",i,j)\n    fe.essence(i,j)'

In [18]:
"""category_y = [["入線順位","複勝オッズ下限"]]
for i,j in category_y:
    print("processing:",i)
    fe.categorical(i,j)"""

'category_y = [["入線順位","複勝オッズ下限"]]\nfor i,j in category_y:\n    print("processing:",i)\n    fe.categorical(i,j)'

In [19]:
"""fe.ranking_and_number("レースID(新)")
train_data = fe.df #これでranking_and_numberが反映される"""

'fe.ranking_and_number("レースID(新)")\ntrain_data = fe.df #これでranking_and_numberが反映される'

In [20]:
bks = [["場所","距離","芝・ダ"],["前走場所","前距離","前芝・ダ"]]
for b,k,s in bks:
    print("processing:",b,k,s)
    fe.basyo_kyori_sibadart(b,k,s)
    train_data = fe.df

processing: 場所 距離 芝・ダ
processing: 前走場所 前距離 前芝・ダ


In [21]:
train_data.head()

,レースID(新),場所,芝・ダ,距離,クラス名,馬場状態,出走頭数,騎手,枠番,性別,年齢,キャリア,斤量,馬体重,馬体重増減,間隔,休み明け〜戦目,種牡馬,父タイプ名,所属,調教師,入線順位,確定着順,着差,単勝オッズ,複勝オッズ下限,複勝オッズ上限,複勝配当,前走レースID(新),前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走出走頭数,替,前騎手,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走上り3F順,前走RPCI,前PCI,前好走,前走入線順位,前走確定着順,前走着差タイム,前走PCI3,前走単勝オッズ,場所距離芝ダ,前走場所距離芝ダ
0,201312230605091207,2,2,1200,2,3,16,497,4,2,3,8,56,494,4.0,3.0,5,477,8,1,387,1,1,0.0,1.7,1.1,1.1,110,201311300605011206,2,2,1200.0,2,3,16.0,0,87,3.0,3,56,34.8,34.5,5.0,48.9,50.9,0,2.0,2.0,0.1,51.47,1.9,76,118
1,201312230605091203,2,2,1200,2,3,16,142,2,2,3,12,56,466,0.0,3.0,4,712,8,0,260,2,2,0.1,39.1,4.9,10.8,570,201311300605011215,2,2,1200.0,2,3,16.0,1,142,8.0,4,56,35.6,34.2,2.0,48.9,54.1,0,7.0,7.0,0.6,51.47,41.3,76,118
2,201312230605091205,2,2,1200,2,3,16,283,3,1,3,14,54,476,-2.0,3.0,5,732,6,1,123,3,3,0.3,12.8,2.1,4.3,260,201311300605011201,2,2,1200.0,2,3,16.0,1,283,1.0,2,54,35.0,34.6,7.0,48.9,51.2,0,4.0,4.0,0.4,51.47,5.1,76,118
3,201312230605091208,2,2,1200,2,3,16,135,4,1,4,15,55,460,-6.0,5.0,9,400,6,0,249,4,4,0.4,5.0,1.4,2.2,0,201311160805051214,3,2,1400.0,2,3,14.0,0,39,8.0,3,55,36.3,34.1,7.0,56.6,56.5,0,5.0,5.0,0.4,58.67,4.7,76,202
4,201312230605091211,2,2,1200,2,3,16,316,6,2,3,16,56,536,6.0,5.0,13,774,8,1,95,5,5,0.5,13.0,2.5,5.1,0,201311160505051203,14,2,1400.0,2,3,15.0,1,316,2.0,4,56,37.2,32.8,1.0,58.6,63.4,0,8.0,8.0,0.4,60.23,62.8,76,55


## ID変換  

In [22]:
train_data = train_data.set_index("レースID(新)")


In [23]:
columns_name = ["レースID(新)","場所","芝・ダ","距離","クラス名","馬場状態","出走頭数","騎手","枠番","性別","年齢","キャリア","斤量","馬体重","馬体重増減","間隔","休み明け〜戦目","種牡馬",
                "父タイプ名","所属","調教師","入線順位","確定着順","着差","単勝オッズ","複勝オッズ下限","複勝オッズ上限","複勝配当","場所距離芝ダ",
                "前走レースID(新)","前走場所","前芝・ダ","前距離","前クラス名","前走馬場状態","前走出走頭数","替","前騎手","前走枠番","前走脚質","前走斤量","前走Ave-3F",
                "前走上り3F","前走上り3F順","前走PCI3","前走RPCI","前PCI","前好走","前走入線順位","前走確定着順","前走着差タイム","前走単勝オッズ", "前走場所距離芝ダ",
                "前走レースID(新)1","前走場所1","前芝・ダ1","前距離1","前クラス名1","前走馬場状態1","前走出走頭数1","替1","前騎手1","前走枠番1","前走脚質1","前走斤量1","前走Ave-3F1",
                "前走上り3F1","前走上り3F順1","前走PCI31","前走RPCI1","前PCI1","前好走1","前走入線順位1","前走確定着順1","前走着差タイム1","前走単勝オッズ1", "前走場所距離芝ダ1",
                "前走レースID(新)2","前走場所2","前芝・ダ2","前距離2","前クラス名2","前走馬場状態2","前走出走頭数2","替2","前騎手2","前走枠番2","前走脚質2","前走斤量2","前走Ave-3F2",
                "前走上り3F2","前走上り3F順2","前走PCI32","前走RPCI2","前PCI2","前好走2","前走入線順位2","前走確定着順2","前走着差タイム2","前走単勝オッズ2", "前走場所距離芝ダ2"
               ]

In [24]:
train_data = train_data.to_dict(orient="index")

In [25]:
import time
t1 = 0
t2 = 0

In [27]:
base_df = pd.DataFrame(columns=columns_name)
e2 = []
for i in range(2014,2019+1):
    id_df = pd.read_csv(data_dir+"id"+f"{i}"+".csv",encoding="shift-jis",names=("日付データ","今走","前1走","前2走"))
    id_df = id_df.values.tolist()
    res_df = pd.DataFrame(columns=columns_name)
    res_res_df = pd.DataFrame(columns=columns_name)
    for j in range(len(id_df)):
        if j%1000==0:
            t2 = time.time()
            print("processing:",i,j,t2-t1)
            t1 = t2
        for k in range(3):
            if k == 0: #今走の場合のみ、今走のデータを取り出す
                try:
                    race_id = int(id_df[j][k+1])
                    res = pd.Series({"レースID(新)":race_id})
                    res = res.append(pd.Series(train_data[race_id]))
                    continue
                except: #ここでエラー出たらもうしょうがないので飛ばす
                    try:
                        race_id = str(id_df[j][k+1])
                        res = pd.Series({"レースID(新)":race_id})
                        res = res.append(pd.Series(train_data[race_id]))
                        continue
                    except:
                        break
            race_id = id_df[j][k+1] #str or int or nan
            if race_id != np.nan:
                try: #intでtry
                    race_data = pd.Series(train_data[int(race_id)])
                    race_data = race_data[["前走レースID(新)","前走場所","前芝・ダ","前距離","前クラス名","前走馬場状態","前走出走頭数","替","前騎手","前走枠番","前走脚質","前走斤量","前走Ave-3F",
                "前走上り3F","前走上り3F順","前走PCI3","前走RPCI","前PCI","前好走","前走入線順位","前走確定着順","前走着差タイム","前走単勝オッズ", "前走場所距離芝ダ"]]
                    race_data = race_data.rename(index=lambda s: s+str(k))
                    res = res.append(race_data) # 必要なデータだけで良い
                except:
                    try: #文字列型でtry
                        race_data = pd.Series(train_data[race_id])
                        race_data = race_data[["前走レースID(新)","前走場所","前芝・ダ","前距離","前クラス名","前走馬場状態","前走出走頭数","替","前騎手","前走枠番","前走脚質","前走斤量","前走Ave-3F",
                    "前走上り3F","前走上り3F順","前走PCI3","前走RPCI","前PCI","前好走","前走入線順位","前走確定着順","前走着差タイム","前走単勝オッズ", "前走場所距離芝ダ"]]
                        race_data = race_data.rename(index=lambda s: s+str(k))
                        res = res.append(race_data) # 必要なデータだけで良い
                    except: #新馬戦時は前データが無いことになるのでここで結構弾かれる
                        break
            else: #レースIDが欠損してた場合
                break
        
        res.name = j # res:馬毎
        res_df = res_df.append(res) # res_df:年毎
        if j%500==0:
            res_res_df = pd.concat([res_res_df,res_df])
            res_df = pd.DataFrame(columns=columns_name)
    #最後の処理(i%500でひっかからない最後)
    res_res_df = pd.concat([res_res_df,res_df])
    base_df = pd.concat([base_df,res_res_df])
    
    #年ごとにcsv出力
    res_res_df.to_csv(data_dir+str(i)+"data"+tail+".csv",index=False,encoding="shift-jis")
    #df形式でも出力
    with open(pickle_dir+str(i)+"data"+tail+'.pickle', mode='wb') as fp:
        pickle.dump(res_res_df, fp)

processing: 2014 0 114.51781821250916
processing: 2014 1000 26.86150860786438
processing: 2014 2000 25.559717655181885
processing: 2014 3000 26.043350219726562
processing: 2014 4000 25.416075944900513
processing: 2014 5000 26.87612771987915
processing: 2014 6000 27.8904230594635
processing: 2014 7000 31.689325094223022
processing: 2014 8000 26.424344539642334
processing: 2014 9000 28.23132872581482
processing: 2014 10000 28.189714908599854
processing: 2014 11000 27.030417442321777
processing: 2014 12000 26.301666259765625
processing: 2014 13000 27.729329586029053
processing: 2014 14000 28.4130220413208
processing: 2014 15000 27.08904218673706
processing: 2014 16000 29.399386167526245
processing: 2014 17000 28.611496210098267
processing: 2014 18000 28.746099710464478
processing: 2014 19000 27.263123035430908
processing: 2014 20000 29.4044029712677
processing: 2014 21000 28.328108072280884
processing: 2014 22000 27.796152591705322
processing: 2014 23000 29.45924735069275
processing: 2014

In [28]:
with open(pickle_dir+tail+"base_df"+'.pickle', mode='wb') as fp:
        pickle.dump(base_df, fp)